In [3]:
import pandas as pd

df = pd.read_csv("/content/Participant_Combined.csv")
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
df['gender_numeric'] = le.fit_transform(df['gender'])
df['environment_numeric'] = le.fit_transform(df['environment'])


df = df.drop('subject_id', axis=1)
df= df.drop('id',axis=1)
df= df.drop('unnamed: 0',axis=1)
df=df.drop('travel_time',axis=1)
df=df.drop('environment',axis=1)
df=df.drop('gender',axis=1)
# You manually define behavioral columns
behavior_columns = ['pre_saa','post_saa','pre_cort','post_cort','total_reward','patch_switching_freq']

# The label column
label_column = 'trait_anx_level'


# Automatically get EEG columns = all columns except behavioral + label
eeg_columns = [col for col in df.columns if col not in behavior_columns + [label_column]]


In [4]:
non_numeric_cols = df[eeg_columns].select_dtypes(exclude=["number"]).columns
print("Non-numeric EEG columns:", list(non_numeric_cols))


Non-numeric EEG columns: []


In [5]:
import torch
from torch.utils.data import TensorDataset, DataLoader

# Convert EEG and Behavioral Data to tensors
eeg_data = torch.tensor(df[eeg_columns].values, dtype=torch.float32)
behavior_data = torch.tensor(df[behavior_columns].values, dtype=torch.float32)
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
df['trait_anx_level'] = le.fit_transform(df['trait_anx_level'])  # overwrite or use a new column if you prefer

labels = torch.tensor(df['trait_anx_level'].values, dtype=torch.long)


In [6]:
from sklearn.model_selection import train_test_split

# Convert tensors to numpy for splitting
eeg_np = eeg_data.numpy()
behavior_np = behavior_data.numpy()
labels_np = labels.numpy()

# Split
eeg_train, eeg_test, behavior_train, behavior_test, labels_train, labels_test = train_test_split(
    eeg_np, behavior_np, labels_np, test_size=0.2, random_state=42, stratify=labels_np
)

# Convert back to tensors
eeg_train = torch.tensor(eeg_train, dtype=torch.float32)
eeg_test = torch.tensor(eeg_test, dtype=torch.float32)

behavior_train = torch.tensor(behavior_train, dtype=torch.float32)
behavior_test = torch.tensor(behavior_test, dtype=torch.float32)

labels_train = torch.tensor(labels_train, dtype=torch.long)
labels_test = torch.tensor(labels_test, dtype=torch.long)


In [7]:
train_dataset = TensorDataset(eeg_train, behavior_train, labels_train)
test_dataset = TensorDataset(eeg_test, behavior_test, labels_test)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32)


In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split
import numpy as np

# ===== 1. Data Preparation (assuming you already have these) =====

# Convert to NumPy for splitting
eeg_np = eeg_data.numpy()
behavior_np = behavior_data.numpy()
labels_np = labels.numpy()

# Split into train and test
eeg_train, eeg_test, behavior_train, behavior_test, labels_train, labels_test = train_test_split(
    eeg_np, behavior_np, labels_np, test_size=0.2, random_state=42, stratify=labels_np
)

# Convert back to tensors
eeg_train = torch.tensor(eeg_train, dtype=torch.float32)
eeg_test = torch.tensor(eeg_test, dtype=torch.float32)

behavior_train = torch.tensor(behavior_train, dtype=torch.float32)
behavior_test = torch.tensor(behavior_test, dtype=torch.float32)

labels_train = torch.tensor(labels_train, dtype=torch.long)
labels_test = torch.tensor(labels_test, dtype=torch.long)

# DataLoaders
train_dataset = TensorDataset(eeg_train, behavior_train, labels_train)
test_dataset = TensorDataset(eeg_test, behavior_test, labels_test)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32)

# ===== 2. Define Multi-Branch Model =====

eeg_dim = len(eeg_columns)
behavior_dim = len(behavior_columns)

class MultiBranchEEGBehaviorModel(nn.Module):
    def __init__(self, eeg_dim, behavior_dim):
        super().__init__()

        self.eeg_branch = nn.Sequential(
            nn.Linear(eeg_dim, 32),
            nn.ReLU(),
            nn.Dropout(0.2)
        )

        self.behavior_branch = nn.Sequential(
            nn.Linear(behavior_dim, 16),
            nn.ReLU(),
            nn.Dropout(0.2)
        )

        self.classifier = nn.Sequential(
            nn.Linear(32 + 16, 32),
            nn.ReLU(),
            nn.Linear(32, 2)  # 2 classes: stay/leave
        )

    def forward(self, eeg_input, behavior_input):
        eeg_out = self.eeg_branch(eeg_input)
        behavior_out = self.behavior_branch(behavior_input)
        combined = torch.cat((eeg_out, behavior_out), dim=1)
        return self.classifier(combined)

# ===== 3. Train the Model =====

model = MultiBranchEEGBehaviorModel(eeg_dim, behavior_dim)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    correct = 0
    total = 0

    for eeg_batch, behavior_batch, label_batch in train_loader:
        optimizer.zero_grad()
        output = model(eeg_batch, behavior_batch)
        loss = criterion(output, label_batch)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        predicted = torch.argmax(output, dim=1)
        correct += (predicted == label_batch).sum().item()
        total += label_batch.size(0)

    accuracy = correct / total * 100
    print(f"Epoch {epoch+1}/{num_epochs} | Loss: {total_loss:.4f} | Accuracy: {accuracy:.2f}%")

# ===== 4. Inference on Test Set =====

model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for eeg_batch, behavior_batch, label_batch in test_loader:
        output = model(eeg_batch, behavior_batch)
        predicted = torch.argmax(output, dim=1)
        all_preds.append(predicted)
        all_labels.append(label_batch)

# Combine predictions and labels
y_pred = torch.cat(all_preds)
y_true = torch.cat(all_labels)

# Print class distribution
print(f"\nPredicted class distribution: {torch.bincount(y_pred)}")


Epoch 1/10 | Loss: 52.3432 | Accuracy: 50.00%
Epoch 2/10 | Loss: 21.8098 | Accuracy: 62.50%
Epoch 3/10 | Loss: 51.4606 | Accuracy: 46.25%
Epoch 4/10 | Loss: 39.9381 | Accuracy: 46.25%
Epoch 5/10 | Loss: 43.2914 | Accuracy: 53.75%
Epoch 6/10 | Loss: 32.8253 | Accuracy: 51.25%
Epoch 7/10 | Loss: 35.9723 | Accuracy: 47.50%
Epoch 8/10 | Loss: 21.8807 | Accuracy: 56.25%
Epoch 9/10 | Loss: 26.7386 | Accuracy: 58.75%
Epoch 10/10 | Loss: 39.3208 | Accuracy: 47.50%

Predicted class distribution: tensor([ 0, 20])


In [11]:
with torch.no_grad():
    all_preds = []
    for eeg_batch, behavior_batch, _ in test_loader:
        outputs = model(eeg_batch, behavior_batch)
        preds = torch.argmax(outputs, dim=1)
        all_preds.append(preds)

    predicted_classes = torch.cat(all_preds)

# Count predictions
counts = torch.bincount(predicted_classes)
accuracy = correct / total * 100
print(f"\nTest Accuracy: {accuracy:.2f}%")
# If you used LabelEncoder, you can map them back:
print(f"Predicted 'leave': {counts[0].item()}")
print(f"Predicted 'stay' : {counts[1].item()}")



Test Accuracy: 47.50%
Predicted 'leave': 0
Predicted 'stay' : 20
